In [1]:
#encoding:utf8
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import keras
from keras.datasets import cifar10
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPool2D
from keras.optimizers import RMSprop,SGD
from keras.models import load_model
from keras.models import model_from_json
import scipy.io as sio
import tensorflow as tf

from flask import Flask, render_template, request
from scipy.misc import imsave, imread, imresize
import re
import sys
import os
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.applications.resnet50 import preprocess_input, decode_predictions
from getdata import load
from keras.models import Model
from keras.layers import Input, Dense
from keras import backend as K
# K.set_image_dim_ordering('th')

/home/curie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def preprocess_img(img_path):
    img = image.load_img(img_path,target_size=(244,244))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis = 0)
    x = preprocess_input(x)
    return x
# def predict_top_labels(preds,top_num):
#     print('Predicted:', decode_predictions(preds, top=top_num)[0])
#     return

In [3]:
# base_model = ResNet50(weights='imagenet')
base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(244,244,3))
# base_model = ResNet50(include_top=False, weights=None, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False
# model = Sequential()
# model.add(base_model)
# model.add(Flatten())
# model.add(Dense(256,activation='relu'))
# model.add(Dense(input_dim=1, activation='sigmoid',
#             bias_initializer='normal', units=5)) 
# base_model_out = base_model.output
# my_layer = Flatten()(base_model_out)
# my_layer = Dense(256,activation='relu')(my_layer)
# model = Dense(input_dim=1, activation='sigmoid',
#             bias_initializer='normal', units=5)(my_layer) 

# rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# # learning rate should be low, as Resnet is ok
# # model.compile(optimizer=rms, loss='categorical_crossentropy')

# # model.compile(optimizer='rmsprop',
# #http://www.datalearner.com/blog/1051521451493989
# #在多标签分类中，大多使用binary_crossentropy损失而不是通常在多类分类中使用
# # 的categorical_crossentropy损失函数。这可能看起来不合理，但因为每个输出节点都是独立的
# # ，选择二元损失，并将网络输出建模为每个标签独立的bernoulli分布。
# model.compile(optimizer=rms ,
#             loss='binary_crossentropy',
#             metrics=['accuracy'])

# model.summary()

# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [4]:
base_model_out = base_model.output
# my_layer = base_model.add(Flatten())
base_model_out=Flatten(name='my_flatten')(base_model_out)
my_layer = Dense(256,activation='relu')(base_model_out)
predictions = Dense(input_dim=1, activation='sigmoid',
            bias_initializer='normal', units=5)(my_layer) 

rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# learning rate should be low, as Resnet is ok
# model.compile(optimizer=rms, loss='categorical_crossentropy')

# model.compile(optimizer='rmsprop',
#http://www.datalearner.com/blog/1051521451493989
#在多标签分类中，大多使用binary_crossentropy损失而不是通常在多类分类中使用
# 的categorical_crossentropy损失函数。这可能看起来不合理，但因为每个输出节点都是独立的
# ，选择二元损失，并将网络输出建模为每个标签独立的bernoulli分布。
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=rms ,
            loss='binary_crossentropy',
            metrics=['accuracy'])

model.summary()

from keras.utils import plot_model
plot_model(model, to_file='model.png')

# base_model = resnet50 (weights='imagenet', include_top=False)

# # add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# # and a logistic layer -- let's say we have 7 classes
# predictions = Dense(7, activation='softmax')(x) 
# model = Model(inputs=base_model.input, outputs=predictions)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 244, 244, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 250, 250, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 122, 122, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 122, 122, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
img_path = "../pics/woodgirl.jpg"
x = preprocess_img(img_path)
preds = model.predict(x)
print(preds)
# predict_top_labels(preds,3)
# after fitting the model and calculate predictions you have to manually assign the 
# class name to output number without using imported decode_predictions
# https://stackoverflow.com/questions/49259361/valueerror-decode-predictions-expects-a-batch-of-predictions-i-e-a-2d-array
#https://stackoverflow.com/questions/49259361/valueerror-decode-predictions-expects-a-batch-of-predictions-i-e-a-2d-array

[[0.9454877  0.90882546 0.5331726  0.8134395  0.76214784]]


In [6]:
def my_predict(img_path):
    print(img_path)
    x = preprocess_img(img_path)
    preds = model.predict(x)
    print(preds)
#     predict_top_labels(preds,3)

In [8]:
# x_train_batch, y_train_batch = tf.train.shuffle_batch(
#     tensors=[data.train.images, data.train.labels.astype(np.int32)],
#     batch_size=batch_size,
#     capacity=capacity,
#     min_after_dequeue=min_after_dequeue,
#     enqueue_many=enqueue_many,
#     num_threads=8)
# load tfrecord to dataset
import tensorflow as tf
# dataset=tf.contrib.data.TFRecordDataset("train.bin").repeat()

# dataset=tf.data.TFRecordDataset("train.bin").repeat()
filenames = ["../../../Dataset/miml-image-data/processed/train.tfrecord", 
             "../../../Dataset/miml-image-data/processed/test.tfrecord"]
# my_dataset = tf.contrib.data.TFRecordDataset(filenames)
train_dataset = tf.data.TFRecordDataset([filenames[0]])
test_dataset = tf.data.TFRecordDataset([filenames[1]])

In [9]:
print(train_dataset)
print(test_dataset)

<TFRecordDataset shapes: (), types: tf.string>
<TFRecordDataset shapes: (), types: tf.string>


In [2]:
import os
def visit_dir(path):
    li = os.listdir(path)
    for p in li:
        pathname = os.path.join(path,p)
        my_predict(pathname)
#         my_predict(pathname)
    
base_path = "../pics/nature"
visit_dir(base_path)

NameError: name 'my_predict' is not defined

In [21]:
def parse_tf(example):
    dics = {}
    dics['label_1'] = tf.FixedLenFeature(shape=[5],dtype=tf.int64)
    dics['label_2'] = tf.FixedLenFeature(shape=[5], dtype=tf.int64)
    dics['label_3'] = tf.FixedLenFeature(shape=[5], dtype=tf.int64)
    dics['label_4'] = tf.FixedLenFeature(shape=[5], dtype=tf.int64)
    dics['label_5'] = tf.FixedLenFeature(shape=[5], dtype=tf.int64)
    dics['raw_image'] = tf.FixedLenFeature(shape=[],dtype=tf.string)
    parsed = tf.parse_single_example(example,features=dics)
    image = tf.decode_raw(parsed['raw_image'],out_type=tf.uint8)
    image = tf.reshape(image,shape=[224,224,3])
    image = tf.image.per_image_standardization(image)
    label_1 = parsed['label_1']
    label_2 = parsed['label_2']
    label_3 = parsed['label_3']
    label_4 = parsed['label_4']
    label_5 = parsed['label_5']

    label_1 = tf.cast(label_1,tf.int32)
    label_2 = tf.cast(label_2, tf.int32)
    label_3 = tf.cast(label_3, tf.int32)
    label_4 = tf.cast(label_4, tf.int32)
    label_5 = tf.cast(label_5, tf.int32)

    return image,label_1,label_2,label_3,label_4,label_5

In [11]:
# https://stackoverflow.com/questions/42184863/how-do-you-make-tensorflow-keras-fast-with-a-tfrecord-dataset
from keras import backend as K
sess = tf.Session()
K.set_session(sess)
x_train, x_test, y_train, y_test = load()

x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

x_train /= 255
x_test /= 255
# train_dataset = train_dataset.map(parse_tf)
# train_dataset = train_dataset.batch(16).repeat(1)
# train_iter = train_dataset.make_one_shot_iterator()

# test_dataset = test_dataset.map(parse_tf)
# test_dataset = test_dataset.batch(16).repeat(1)
# test_iter = test_dataset.make_one_shot_iterator()

In [13]:
batch_size = 32
epochs = 200
history = model.fit(x_train,y_train,
                epochs=epochs,
                batch_size=batch_size)
loss,accuracy = model.evaluate(x_test,y_test)
print('Test loss: ',loss)                
print('Test accuracy: ',accuracy)
print("多标签模型训练完毕")
model_path = "../model_path/multi-label.h5"
model.save(model_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

ValueError: Error when checking input: expected input_1 to have shape (244, 244, 3) but got array with shape (3, 100, 100)